# How to

From http://dl.turkunlp.org/wikibert/wikibert-base-sk-cased/ download:
 - `wikibert-base-sk-cased.tar.gz` (extract to `'./models/'`)
 - `vocab.txt` (move to `'./models/wikibert-base-sk-cased/'`)
 
Run jupyter notebook in virtual enviroment. [tutorial](https://janakiev.com/blog/jupyter-virtual-envs/)
 
`Summarizer` package:
 - installation: `pip install bert-extractive-summarizer`
 - requirements
     ```
     spacy==2.1.3
     transformers==2.2.2
     neuralcoref
     ``` 
 
Other requirements:
 - `pytorch-transformers` (pip install)
 


In [1]:
from transformers import *

# Load model, model config and tokenizer via Transformers
custom_config = AutoConfig.from_pretrained('models/wikibert-base-sk-cased')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('models/wikibert-base-sk-cased')
custom_model = AutoModel.from_pretrained('models/wikibert-base-sk-cased', config=custom_config)


from summarizer import Summarizer
#package: https://pypi.org/project/bert-extractive-summarizer/

In [4]:
body = '''Centy zatiaľ Slovákom v peňaženkách ostanú . \
Na dnešnej konferencii venovanej prvému roku s eurom to povedal \
guvernér Národnej banky Slovenska ( NBS ) Ivan Šramko . Podľa prieskumu , \
ktorý realizovalo Zastúpenie Európskej komisie na Slovensku v októbri tohto roku , \
vyplýva , že spotrebiteľom aj obchodníkom z euromincí najviac " prekážajú " \
práve jedno - a dvojcentové mince . " Rušenie centov ako téma neexistuje . \
Občania si zvykli , pokiaľ niekto nemá záujem ich používať , tak ich nepoužíva , \
" konštatoval Šramko . Prezident Zväzu obchodu a cestovného ruchu Pavol Konštiak si \
naopak myslí , že by bolo vhodné , aby sa začalo diskutovať aspoň o možnosti zaokrúhľovania \
cien na 5 centov . " Zákon o zaokrúhľovaní na päťcentovky by výrazne pomohol spotrebiteľom , \
ale aj obchodníkom , " spresnil . Centy by tak nezmizli , používal by ich len ten , \
kto chce a ceny by boli podľa Konštiaka prehľadnejšie .'''

model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)

result = model(body)


print('SUMMARIZATION:',''.join(result))

SUMMARIZATION: Centy zatiaľ Slovákom v peňaženkách ostanú . Podľa prieskumu , ktorý realizovalo Zastúpenie Európskej komisie na Slovensku v októbri tohto roku , vyplýva , že spotrebiteľom aj obchodníkom z euromincí najviac " prekážajú " práve jedno - a dvojcentové mince . "


In [5]:
import torch

tokenizer = custom_tokenizer
tokenized_ids = tokenizer.encode('Z tejto vety chceme dostat embeddingy.')

# Convert the list of IDs to a tensor of IDs 
tokenized_ids = torch.LongTensor(tokenized_ids)
# Print the IDs
print('tokenized IDs:', tokenized_ids)

custom_model.eval()

# unsqueeze IDs to get batch size of 1 as added dimension
tokenized_ids = tokenized_ids.unsqueeze(0)
print('Unsqeezed IDs size:', tokenized_ids.size())

with torch.no_grad():
    out = custom_model(input_ids=tokenized_ids)


# the tuple contains three elements
# we only want the hidden_states
hidden_states = out[2]
#print(len(hidden_states))

sentence_embedding = torch.mean(hidden_states[-1], dim=1).squeeze()
print('Sentence embeddings:', sentence_embedding)
print('Sentence embeddings size:', sentence_embedding.size())

tokenized IDs: tensor([  102,   120,  1072, 15512,  3942,   215,  1292, 19858, 15571,   198,
        13254, 19874,   107,   103])
Unsqeezed IDs size: torch.Size([1, 14])
Sentence embeddings: tensor([-2.1587e-02,  5.9894e-02,  2.2663e-01,  6.6777e-03, -3.5917e-01,
         8.1850e-01,  1.3710e-01,  1.6127e-01,  1.6782e-01, -2.0125e-01,
         8.2840e-02, -2.0381e-02,  1.5064e-01,  2.4481e-02,  5.2147e-02,
        -4.1739e-02,  1.5603e-01,  7.6994e-02,  5.6781e-02, -1.5501e-01,
        -3.5505e-01,  3.0045e-01, -2.8287e-01,  2.1736e-02,  2.8922e-01,
        -3.4148e-01, -7.6770e-02, -2.5796e-01,  4.9605e-01, -7.9952e-02,
         1.0967e-01, -1.5430e-01,  1.3490e-01, -8.8833e-02,  2.7862e-02,
         1.9966e-02,  7.8283e-02,  2.3484e-02, -5.4474e-01,  5.9344e-02,
        -3.3819e-02, -3.6301e-01,  1.6126e-02,  5.2616e-03, -2.1643e-01,
        -1.0642e-01,  8.7343e-02,  1.0169e-01, -3.6877e-01, -8.1257e-02,
         5.2543e-01, -1.5635e-01,  2.1295e-02, -3.4736e-01,  1.5908e-02,
      